# Read Shortcut Data

Read shortcuts from:
1. **Parquet output** - Final deduplicated shortcuts
2. **DuckDB database** - All tables

In [1]:
import duckdb
import pandas as pd
from pathlib import Path

In [22]:
# Configuration
DISTRICT = "Burnaby"  # or "All_Vancouver"
PARQUET_FILE = Path(f"../../../data/{DISTRICT}_shortcuts")
DB_FILE = Path(f"../../../data/{DISTRICT}.db")

## 1. Read from Parquet

In [23]:
shortcuts_df = duckdb.read_parquet(str(PARQUET_FILE)).df()
print(f"Loaded {len(shortcuts_df):,} shortcuts")
shortcuts_df.head()

Loaded 4,173,086 shortcuts


,from_edge,to_edge,cost,via_edge,inside,cell
0,14478,8381,157.644813,8365,1,590690563070623743
1,14478,10552,299.672726,15753,-2,590690563070623743
2,14478,19533,97.225200,4133,0,599697731186851839
3,14478,19780,198.349050,19802,1,590690563070623743
4,14478,23455,276.364837,192,-2,590690563070623743


In [24]:
df = shortcuts_df
df[(df['from_edge'] == 204) & (df['to_edge'] == 2656)]

,from_edge,to_edge,cost,via_edge,inside,cell


In [25]:
df[(df['from_edge'] == 204) & (df['to_edge'] == 777)]

,from_edge,to_edge,cost,via_edge,inside,cell


In [26]:
df[(df['from_edge'] == 204) & (df['to_edge'] == 2652)]

,from_edge,to_edge,cost,via_edge,inside,cell


In [27]:
print(f"Cost range: {shortcuts_df['cost'].min():.2f} to {shortcuts_df['cost'].max():.2f}")
print(f"Unique from_edge: {shortcuts_df['from_edge'].nunique():,}")
print(f"Unique to_edge: {shortcuts_df['to_edge'].nunique():,}")

Cost range: 0.01 to 377.59
Unique from_edge: 35,217
Unique to_edge: 35,217


## 2. Read from DuckDB

In [28]:
con = duckdb.connect(str(DB_FILE), read_only=True)

# List tables
for t in con.execute("SHOW TABLES").fetchall():
    count = con.execute(f"SELECT count(*) FROM {t[0]}").fetchone()[0]
    print(f"{t[0]}: {count:,} rows")

dataset_info: 4 rows
edges: 35,217 rows
shortcuts: 4,173,086 rows


In [29]:
# Edges table
con.execute("SELECT * FROM edges LIMIT 5").df()

,id,from_cell,to_cell,lca_res,length,cost,geometry
0,31852,644733726876424710,644733726875794148,8,41.599,4.99188,LINESTRING (-122.96475219726562 49.23262405395...
1,31853,644733726876424710,644733726560578696,5,105.943,19.06974,LINESTRING (-122.96475219726562 49.23262405395...
2,31854,644733726876424710,644733726875755904,8,154.765,18.57180,LINESTRING (-122.96475219726562 49.23262405395...
3,31855,644733694717892692,644733694717909313,10,49.629,8.93322,LINESTRING (-122.98440551757812 49.27865982055...
4,31856,644733694719239020,644733694718810467,8,48.707,8.76726,"LINESTRING (-122.9843978881836 49.27685546875,..."


In [10]:
# Shortcuts table
con.execute("SELECT * FROM shortcuts LIMIT 5").df()

,from_edge,to_edge,cost,via_edge,inside,cell
0,4840,3821,69.597818,369,0,0
1,4840,4758,113.046633,369,0,0
2,4842,1803,132.233683,369,0,0
3,4842,3832,68.508401,3426,0,0
4,4842,3952,182.199613,369,0,0


In [11]:
# Cost statistics
con.execute("""
    SELECT MIN(cost), AVG(cost), MEDIAN(cost), MAX(cost)
    FROM shortcuts
""").df()

,"min(""cost"")","avg(""cost"")","median(""cost"")","max(""cost"")"
0,0.045675,39.951896,27.743942,475.348886


In [12]:
# Top connected edges
con.execute("""
    SELECT from_edge, COUNT(*) as destinations
    FROM shortcuts GROUP BY from_edge
    ORDER BY destinations DESC LIMIT 10
""").df()

,from_edge,destinations
0,2696,704
1,5660,704
2,2343,704
3,3052,704
4,1074,704
5,1073,703
6,1069,703
7,4023,700
8,2399,681
9,2400,681


In [13]:
con.close()